- All non-negative coefficients
- Bounds for Intercept depend on no_intercept flag also
- Specific order for some coefficients
- Min percentage by which the coefficients (of ordered features) differ

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from scipy.optimize import lsq_linear

# User inputs

In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
no_intercept = True

In [4]:
target = 'SalePrice'

In [5]:
# Features in the expected ascending order of coefficients
features = ['PoolArea', 'LotArea', 'TotalBsmtSF', 'GarageArea', 'GrLivArea']
features

['PoolArea', 'LotArea', 'TotalBsmtSF', 'GarageArea', 'GrLivArea']

In [6]:
# Min percentage gaps between successive (ordered) features
min_gap_pct = [None, 0.25, 0.25, 0.25, 0.25]

# Constraints

In [7]:
# Initialize coefficients
len_coeffs = len(features) + 1
coeffs = list(np.zeros(len_coeffs))
print("Initialized coefficients:", coeffs)

Initialized coefficients: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [8]:
# Put constraints 
min_con = list(np.zeros(len_coeffs))

max_con = [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
if no_intercept:
    max_con[0] = 0.0001

print("Minimum constraints:", min_con)
print("Maximum constraints:", max_con)

Minimum constraints: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Maximum constraints: [0.0001, inf, inf, inf, inf, inf]


# Model

y = X0 + X1*PoolArea + (1.25*X1+X2)*LotArea + (1.25*(1.25*X1+X2)+X3)*TotalBsmtSF + (1.25*(1.25*(1.25*X1+X2)+X3)+X4)*GarageArea + (1.25*(1.25*(1.25*(1.25*X1+X2)+X3)+X4)+X5)*GrLivArea

y = X0 + X1*(PoolArea+1.25*LotArea+1.25*1.25*TotalBsmtSF+1.25*1.25*1.25*GarageArea+1.25*1.25*1.25*1.25*GrLivArea) + X2*(LotArea+1.25*TotalBsmtSF+1.25*1.25*GarageArea+1.25*1.25*1.25*GrLivArea) + X3*(TotalBsmtSF+1.25*GarageArea+1.25*1.25*GrLivArea) + X4*(GarageArea+1.25*GrLivArea) + X5*GrLivArea

In [9]:
# Feature engineer   
X = df[features].copy()
X['F1'] = X[features[0]] + (1+min_gap_pct[1])*X[features[1]] + (1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[2]] + (1+min_gap_pct[3])*(1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[3]] + (1+min_gap_pct[4])*(1+min_gap_pct[3])*(1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[4]]
X['F2'] = X[features[1]] + (1+min_gap_pct[2])*X[features[2]] + (1+min_gap_pct[3])*(1+min_gap_pct[2])*X[features[3]] + (1+min_gap_pct[4])*(1+min_gap_pct[3])*(1+min_gap_pct[2])*X[features[4]]
X['F3'] = X[features[2]] + (1+min_gap_pct[3])*X[features[3]] + (1+min_gap_pct[4])*(1+min_gap_pct[3])*X[features[4]]
X['F4'] = X[features[3]] + (1+min_gap_pct[4])*X[features[4]]
X['F5'] = X[features[4]]
X = X.drop(features, axis=1)

In [10]:
# Convert independent variables to a matrix
X = X.values

# Add an array of ones to act as intercept coefficient
ones = np.ones(X.shape[0])
# Combine array of ones and indepedent variables
X = np.concatenate((ones[:, np.newaxis], X), axis=1)
X

array([[1.00000000e+00, 1.71451172e+04, 1.37160938e+04, 4.21287500e+03,
        2.68550000e+03, 1.71000000e+03],
       [1.00000000e+00, 1.79513672e+04, 1.43610938e+04, 3.80887500e+03,
        2.03750000e+03, 1.26200000e+03],
       [1.00000000e+00, 2.10478516e+04, 1.68382812e+04, 4.47062500e+03,
        2.84050000e+03, 1.78600000e+03],
       ...,
       [1.00000000e+00, 1.93075781e+04, 1.54460625e+04, 5.12325000e+03,
        3.17700000e+03, 2.34000000e+03],
       [1.00000000e+00, 1.69312109e+04, 1.35449688e+04, 3.06237500e+03,
        1.58750000e+03, 1.07800000e+03],
       [1.00000000e+00, 1.79892188e+04, 1.43913750e+04, 3.56350000e+03,
        1.84600000e+03, 1.25600000e+03]])

In [11]:
# Convert target variable to a matrix
y = df[target].values
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [12]:
# Run optimization
results = lsq_linear(X, y, bounds=(min_con, max_con), lsmr_tol='auto')
print("Results:\n", results)

Results:
  active_mask: array([-1, -1,  0,  0, -1, -1])
        cost: 1632714484523.3323
         fun: array([-16896.90744914,  -7957.60201359, -19809.3859197 , ...,
       -33689.98288755,  -2275.63685795,  15015.20132767])
     message: 'The relative change of the cost function is less than `tol`.'
         nit: 32
  optimality: 8.57542184369608e-05
      status: 2
     success: True
           x: array([7.05776931e-51, 5.01332938e-25, 1.59857576e-01, 4.49599077e+01,
       1.78249748e-24, 3.88652663e-31])


In [13]:
if results.success:
    # Transform the coefficients back to the context of original features 
    coeffs[0] = results.x[0]
    coeffs[1] = results.x[1]
    coeffs[2] = (1+min_gap_pct[1])*results.x[1] + results.x[2]
    coeffs[3] = (1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3]
    coeffs[4] = (1+min_gap_pct[3])*((1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3]) + results.x[4]
    coeffs[5] = (1+min_gap_pct[4])*((1+min_gap_pct[3])*((1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3]) + results.x[4]) + results.x[5]
    print("Final Coefficients (including intercept):", coeffs)
else:
    print("Convergence was not achieved!")

Final Coefficients (including intercept): [7.057769312587204e-51, 5.013329381815313e-25, 0.15985757613047077, 45.15972964604039, 56.44966205755048, 70.5620775719381]
